In [132]:
from main import DocCorpus
from corpus import Corpus
import ipywidgets as widgets
from IPython.display import display
from IPython.display import HTML
from datetime import datetime

In [133]:
corpuscharger = Corpus("TestCorpus")
c = corpuscharger.load('biology.pkl')
init = c.construire_vocabulaire()

In [110]:
qWidget = widgets.Text(value = '', placeholder ='Entrez des mot-clefs :')
button = widgets.Button(description='Recherche')
output = widgets.Output()

def recherche_et_affichage(query):
    res_recherche = c.recherche(query)
    html_output = ""
    for resultat in res_recherche.items():
        index_doc = resultat[0]
        doc = c.id2doc[index_doc]
        html_output += f"<p><b>Document : </b> {doc.titre}</p>"
        html_output += f"<p><b>Auteur : </b> {doc.auteur}</p>"
        html_output += f"<p><b>Date : </b> {doc.date}</p>"
        html_output += f"<p><b>Source : </b> {doc.type}</p>"
        html_output += f"<p><b>Contenu : </b> {doc.texte}</p>"
        html_output += f"<p><b>URL : </b><a href='{doc.url}' target='_blank'> {doc.url}</a></p>"
        html_output += "=" * 50 + "<br><br>"
    with output:
        output.clear_output(wait=True)
        display(HTML(html_output))
        
def on_button_click(b):
    output.clear_output()
    recherche_et_affichage(qWidget.value)

button.on_click(on_button_click)
display(qWidget, button, output)

Text(value='', placeholder='Entrez des mot-clefs :')

Button(description='Recherche', style=ButtonStyle())

Output()

In [134]:
qWidget = widgets.Text(value='', placeholder='Entrez des mots-clés :')
auteurWidget = widgets.Text(value='', placeholder='Entrez l\'auteur :')
sourceDropdown = widgets.Dropdown(
    options=['Toutes les sources', 'Reddit', 'Arxiv'],
    value='Toutes les sources',
    description='Source:',
)
datePicker = widgets.DatePicker(
    description='Sélectionnez une date',
    disabled=False
)
button = widgets.Button(description='Recherche')
output = widgets.Output()

def recherche_et_affichage(query, auteur, source, selected_date):
    res_recherche = c.recherche(query)

    html_output = ""
    for resultat in res_recherche.items():
        index_doc = resultat[0]
        doc = c.id2doc[index_doc]

        if (source == 'Toutes les sources' or doc.type == source):
            # Convertir la date en objet datetime au format 'YYYY/MM/DD'
            doc_date = datetime.strptime(doc.date, '%Y/%m/%d').date()

            # Comparer avec la date sélectionnée et l'auteur
            if (selected_date is None or doc_date == selected_date) and (auteur.lower() in doc.auteur.lower()):
                # Formate la date dans le format 'YYYY/MM/DD'
                formatted_date = doc_date.strftime('%Y/%m/%d')

                html_output += f"<p><b>Document : </b> {doc.titre}</p>"
                html_output += f"<p><b>Auteur : </b> {doc.auteur}</p>"
                html_output += f"<p><b>Date : </b> {formatted_date}</p>"
                html_output += f"<p><b>Source : </b> {doc.type}</p>"
                html_output += f"<p><b>Contenu : </b> {doc.texte}</p>"
                html_output += f"<p><b>URL : </b><a href='{doc.url}' target='_blank'> {doc.url}</a></p>"
                html_output += "=" * 50 + "<br><br>"

    with output:
        output.clear_output(wait=True)
        display(HTML(html_output))

def on_button_click(b):
    output.clear_output()
    recherche_et_affichage(qWidget.value, auteurWidget.value, sourceDropdown.value, datePicker.value)

button.on_click(on_button_click)
display(qWidget, auteurWidget, sourceDropdown, datePicker, button, output)

Text(value='', placeholder='Entrez des mots-clés :')

Text(value='', placeholder="Entrez l'auteur :")

Dropdown(description='Source:', options=('Toutes les sources', 'Reddit', 'Arxiv'), value='Toutes les sources')

DatePicker(value=None, description='Sélectionnez une date', step=1)

Button(description='Recherche', style=ButtonStyle())

Output()